In [1]:
library(rstanarm)
library(readr)
library(MASS)
library(broom)
library(dplyr)
library(tidyr)

Loading required package: Rcpp

This is rstanarm version 2.21.1

- See https://mc-stan.org/rstanarm/articles/priors for changes to default priors!

- Default priors may change, so it's safest to specify priors, even if equivalent to the defaults.

- For execution on a local, multicore CPU with excess RAM we recommend calling

  options(mc.cores = parallel::detectCores())


Attaching package: ‘dplyr’


The following object is masked from ‘package:MASS’:

    select


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
responsesall <- read_csv("surveyresponses.csv");


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  When = col_character(),
  Test = col_character(),
  MaskOutCat = col_character(),
  MaskInCat = col_character(),
  SelfAge = col_character(),
  GenderNow = col_character(),
  MaritalStatus = col_character(),
  OccupationStatus = col_character(),
  Race = col_character(),
  Global01 = col_character(),
  MedCond_Heart = col_character(),
  MedCond_HighBP = col_character(),
  MedCond_Immune = col_character(),
  MedCond_Diabetes = col_character(),
  FunctionalHearing = col_character(),
  LanguagePref = col_character(),
  AgeDeaf = col_character(),
  ListeningDevice = col_character()
)
ℹ Use `spec()` for the full column specifications.




In [3]:
responsesall$BMI[is.na(responsesall$BMI)] <- mean(responsesall$BMI,na.rm = TRUE)
responsesall$Severity[is.na(responsesall$Severity)] <- mean(responsesall$Severity,na.rm = TRUE)
responsesall$Global01[is.na(responsesall$Global01)] <- "Unknown"
responsesall$OccupationStatus[is.na(responsesall$OccupationStatus)] <- "Unknown"
responsesall$MedCond_Immune[is.na(responsesall$MedCond_Immune)] <- "Unknown"

In [4]:
responsesall$LangPref2 <- ifelse(responsesall$LanguagePref == "Signed Language",1,0)
responsesall$FuncHear2 <- ifelse(responsesall$FunctionalHearing == "Did not understand what they said",1,0)

In [5]:
responses <- responsesall[responsesall$Test == "Positive" & !is.na(responsesall$Test),]
length(responses$Test)

[1] 95

In [6]:
severityscores<-responses[,c(14:24)]
severity.pca <- prcomp(severityscores)
severityfactors <- t(severity.pca$rotation) %*% t(severityscores)

In [7]:
responses$PCA <- severityfactors[1,] - severityfactors[2,] + severityfactors[3,]
responses$vm <- responses$VoiceOut * responses$MaskOut
responses$vmcat <- responses$VoiceOutCat * responses$MaskOut
responses$vmtot <- responses$VoiceOut * responses$MaskOut + responses$VoiceIn * responses$MaskIn
responses$vtot <- responses$VoiceOut + responses$VoiceIn 
responses$mtot <- responses$MaskOut +  responses$MaskIn

In [8]:
m_stanlm <- stan_lm(PCA~  MedCond_Immune + MedCond_Diabetes +LangPref2 + BMI  + vtot + mtot + vmtot ,data=responses,prior = R2(location = 0.12))


SAMPLING FOR MODEL 'lm' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 5.7e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.57 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 0.227503 seconds (Warm-up)
Chain 1:                0.262439 seconds (Sampling)
Chain 1:                0.4899

In [9]:
summary(m_stanlm,digits=3)
posterior_interval(m_stanlm)


Model Info:
 function:     stan_lm
 family:       gaussian [identity]
 formula:      PCA ~ MedCond_Immune + MedCond_Diabetes + LangPref2 + BMI + vtot + 
	   mtot + vmtot
 algorithm:    sampling
 sample:       4000 (posterior sample size)
 priors:       see help('prior_summary')
 observations: 95
 predictors:   8

Estimates:
                      mean   sd     10%    50%    90% 
(Intercept)          0.782  0.590  0.013  0.794  1.517
MedCond_ImmuneYes    0.777  0.372  0.306  0.771  1.260
MedCond_DiabetesYes  0.117  0.297 -0.263  0.122  0.493
LangPref2            0.163  0.249 -0.152  0.159  0.487
BMI                  0.033  0.014  0.015  0.033  0.052
vtot                 0.025  0.057 -0.047  0.025  0.097
mtot                -0.055  0.105 -0.186 -0.055  0.081
vmtot               -0.056  0.036 -0.103 -0.055 -0.011
sigma                1.076  0.080  0.977  1.071  1.179
log-fit_ratio        0.009  0.072 -0.083  0.007  0.101
R2                   0.140  0.051  0.077  0.136  0.208

Fit Diagnost

,5%,95%
(Intercept),-0.206353373,1.749284346
MedCond_ImmuneYes,0.167174553,1.384875833
MedCond_DiabetesYes,-0.373916813,0.619056474
LangPref2,-0.244958465,0.573415172
BMI,0.009724949,0.057199743
vtot,-0.068955007,0.118709498
mtot,-0.224413870,0.117047974
vmtot,-0.115645610,0.004232906
sigma,0.951843891,1.212141471
log-fit_ratio,-0.105439666,0.127751524


In [10]:
m_stanlm <- stan_lm(PCA~  vtot + mtot + vmtot ,data=responses,prior = R2(location = 0.12))
summary(m_stanlm,digits=3)
posterior_interval(m_stanlm)


SAMPLING FOR MODEL 'lm' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 1.6e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.16 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 0.250294 seconds (Warm-up)
Chain 1:                0.151684 seconds (Sampling)
Chain 1:                0.4019


Model Info:
 function:     stan_lm
 family:       gaussian [identity]
 formula:      PCA ~ vtot + mtot + vmtot
 algorithm:    sampling
 sample:       4000 (posterior sample size)
 priors:       see help('prior_summary')
 observations: 95
 predictors:   4

Estimates:
                mean   sd     10%    50%    90% 
(Intercept)    2.069  0.349  1.627  2.075  2.520
vtot           0.035  0.064 -0.047  0.035  0.116
mtot          -0.079  0.121 -0.234 -0.081  0.076
vmtot         -0.072  0.042 -0.124 -0.072 -0.018
sigma          1.122  0.083  1.020  1.116  1.231
log-fit_ratio  0.014  0.073 -0.080  0.012  0.109
R2             0.076  0.042  0.025  0.070  0.133

Fit Diagnostics:
           mean   sd    10%   50%   90%
mean_PPD 1.770  0.163 1.562 1.771 1.978

The mean_ppd is the sample average posterior predictive distribution of the outcome variable (for details see help('summary.stanreg')).

MCMC diagnostics
              mcse  Rhat  n_eff
(Intercept)   0.008 1.003 1821 
vtot          0.001 0.9

,5%,95%
(Intercept),1.50455872,2.648568909
vtot,-0.07059074,0.139201995
mtot,-0.27739597,0.115785532
vmtot,-0.14144940,-0.004121888
sigma,0.99380431,1.263629894
log-fit_ratio,-0.10377371,0.133198302
R2,0.01635509,0.154489497


In [12]:
m_stanpolr <- stan_polr(as.ordered(Severity)~MedCond_Immune + MedCond_Diabetes +LangPref2 + BMI + vtot + mtot + vmtot,data=responses,prior = R2(0.2), prior_counts = dirichlet(1))
summary(m_stanpolr,digits=3)


SAMPLING FOR MODEL 'polr' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 7.4e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.74 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 0.622478 seconds (Warm-up)
Chain 1:                0.559634 seconds (Sampling)
Chain 1:                1.18


Model Info:
 function:     stan_polr
 family:       ordered [logistic]
 formula:      as.ordered(Severity) ~ MedCond_Immune + MedCond_Diabetes + LangPref2 + 
	   BMI + vtot + mtot + vmtot
 algorithm:    sampling
 sample:       4000 (posterior sample size)
 priors:       see help('prior_summary')
 observations: 95

Estimates:
                      mean   sd     10%    50%    90% 
MedCond_ImmuneYes    0.751  0.534  0.065  0.742  1.442
MedCond_DiabetesYes  1.081  0.457  0.502  1.078  1.659
LangPref2           -0.208  0.366 -0.675 -0.208  0.251
BMI                  0.015  0.021 -0.012  0.015  0.041
vtot                -0.145  0.088 -0.257 -0.144 -0.034
mtot                -0.129  0.164 -0.344 -0.127  0.073
vmtot                0.065  0.054 -0.003  0.065  0.133
1|2                 -1.275  0.863 -2.371 -1.270 -0.161
2|2.63265306122449   0.012  0.859 -1.096  0.009  1.128
2.63265306122449|3   0.230  0.858 -0.870  0.228  1.345
3|4                  1.264  0.862  0.167  1.269  2.372
4|5         